Here I want to write for loops to summarize the differences of variable selection methods (VIF, LSR, high correlation, my own 'guesses', LASSO, divided by 'category' of variable (same day, day before, etc.)) between classification models (random forests, logistic, SVC). 

NB: try only data from 2019? A more complete data set, in terms of predictors (soil moisture, temp, etc.)   

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler, OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingRegressor, StackingRegressor

from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import precision_score, f1_score, accuracy_score, roc_curve, roc_auc_score, make_scorer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

from sklearn.svm import SVR, SVC
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz
from pandas_profiling import ProfileReport
from sklearn.metrics import roc_curve, auc, roc_auc_score

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from statistics import mean


## Intro:
This is a practice with `for` loops to get some sense of how these models are performing. May or may not be the best code/data analysis procedures.

In [233]:
# load in my data:
ascospores = pd.read_csv("2018-19 asco and weather data for modeling.csv", index_col = 0)

In [234]:
ascospores.columns

Index(['SamplerNo', 'Location.x', 'SamplerType', 'ExtractionGroup',
       'qPCR_Plate', 'JDay', 'Date', 'SampleID', 'SsCtMean', 'SsCtSD',
       'SsMean', 'logSsMean', 'Censored', 'SsSD', 'SsScaled', 'SsClean',
       'logSsClean', 'VolumeSampled', 'sporesPCM', 'logSPCM', 'TtCt', 'TtSD',
       'Water.', 'logSsMean_t1', 'Location.y', 'Year', 'MeanWetness',
       'DiffMeanWet', 'MeanWet_1d', 'MaxWetness', 'DiffMaxWet', 'MaxWet_1d',
       'MinWetness', 'DiffMinWet', 'MinWet_1d', 'MeanTemp', 'DiffMeanT',
       'MeanT_1d', 'MaxTemp', 'DiffMaxT', 'MaxT_1d', 'MinTemp', 'DiffMinT',
       'MinT_1d', 'MeanRH', 'DiffMeanRH', 'MeanRH_1d', 'MaxRH', 'DiffMaxRH',
       'MaxRH_1d', 'MinRH', 'DiffMinRH', 'MinRH_1d', 'DiffRH_0d', 'DiffRH_1d',
       'DiffRH_2d', 'MaxDiffRH_2h', 'MaxDiffRH_3h', 'MeanVPD', 'DiffMeanVPD',
       'MeanVPD_1d', 'MaxVPD', 'DiffMaxVPD', 'MaxVPD_1d', 'MinVPD',
       'DiffMinVPD', 'TotalPrecip', 'Precip_1d', 'RainYN', 'MaxRain',
       'MinRain', 'MeanWC', 'DiffMeanWC', 

In [235]:
# remove any lines for which the response variable is missing:
ascospores = ascospores.dropna(subset=['SsMean'])

In [236]:
# select only the 2019 data - this is the most complete data set and will be used for this analysis
asco_2019 = ascospores[ascospores['Year'] == 2019]
asco_2019.shape

(308, 98)

In [237]:
# drop columns that are redundant, not relevant, or contain no useful information
asco_2019 = asco_2019.drop(['Date', 'SamplerType', 'ExtractionGroup','qPCR_Plate', 'SampleID', 'SsCtMean', 
                              'SsCtSD', 'Location.y', 'Censored', 'SsSD', 'SsScaled', 'SsClean', 'logSsClean',
                              'VolumeSampled', 'sporesPCM', 'TtCt', 'TtSD', 'Water.', 'SamplerNo', 'Location.x',
                              'MaxRain', 'MinRain', 'logSPCM'], axis = 1)

In [238]:
print(asco_2019.shape)
asco_2019.columns

(308, 75)


Index(['JDay', 'SsMean', 'logSsMean', 'logSsMean_t1', 'Year', 'MeanWetness',
       'DiffMeanWet', 'MeanWet_1d', 'MaxWetness', 'DiffMaxWet', 'MaxWet_1d',
       'MinWetness', 'DiffMinWet', 'MinWet_1d', 'MeanTemp', 'DiffMeanT',
       'MeanT_1d', 'MaxTemp', 'DiffMaxT', 'MaxT_1d', 'MinTemp', 'DiffMinT',
       'MinT_1d', 'MeanRH', 'DiffMeanRH', 'MeanRH_1d', 'MaxRH', 'DiffMaxRH',
       'MaxRH_1d', 'MinRH', 'DiffMinRH', 'MinRH_1d', 'DiffRH_0d', 'DiffRH_1d',
       'DiffRH_2d', 'MaxDiffRH_2h', 'MaxDiffRH_3h', 'MeanVPD', 'DiffMeanVPD',
       'MeanVPD_1d', 'MaxVPD', 'DiffMaxVPD', 'MaxVPD_1d', 'MinVPD',
       'DiffMinVPD', 'TotalPrecip', 'Precip_1d', 'RainYN', 'MeanWC',
       'DiffMeanWC', 'MeanWC_1d', 'MaxWC', 'DiffMaxWC', 'MaxWC_1d', 'MinWC',
       'DiffMinWC', 'MinWC_1d', 'MeanDP', 'DiffMeanDP', 'MeanDP_1d', 'MaxDP',
       'DiffMaxDP', 'MaxDP_1d', 'MinDP', 'DiffMinDP', 'MinDP_1d',
       'MeanSoilTemp', 'DiffMeanSoilT', 'MeanSoilT_1d', 'MaxSoilTemp',
       'DiffMaxSoilT', 'MaxSoilT_1

This still contains some rendundant/not useful columns ('JDay', 'logSsMean', etc.), but I'll deal with them below

Create train and test data sets:

In [254]:
# only doing trainvalid and test data sets for this analysis due to smaller n
asco_trainvalid, asco_test = train_test_split(asco_2019, test_size = 0.2, random_state = 44)
#asco_train, asco_valid = train_test_split(asco_trainvalid, test_size = 0.2, random_state = 44)

In [255]:
y_trainvalid = asco_trainvalid['SsMean']
y_test = asco_test['SsMean']
y_all = asco_2019['SsMean']

In [256]:
X_trainvalid = asco_trainvalid.drop(['SsMean'], axis = 1)
X_test = asco_test.drop(['SsMean'], axis = 1)
X_all = asco_2019.drop(['SsMean'], axis = 1)

In [51]:
### make data frames for X data sets; export these for use in other programs (R)

X_trainvalid_df = pd.DataFrame(X_trainvalid)
X_test_df = pd.DataFrame(X_test)
X_all_df = pd.DataFrame(X_all)

#X_train_df.columns = new_columns
#X_valid_df.columns = new_columns
#X_test_df.columns = new_columns

X_trainvalid_df.to_csv("~/Desktop/Data/Field Data Analysis/Modeling Data Sets from Python/Variable Selection Data Sets/X_trainvalid.csv")
X_test_df.to_csv("~/Desktop/Data/Field Data Analysis/Modeling Data Sets from Python/Variable Selection Data Sets/X_test.csv")
X_all_df.to_csv("~/Desktop/Data/Field Data Analysis/Modeling Data Sets from Python/Variable Selection Data Sets/X_all.csv")

In [248]:
### 
y100_trainvalid = np.where(y_trainvalid >= 100, 1, 0)
y100_test = np.where(y_test >= 100, 1, 0)
y100_all = np.where(y_all >= 100, 1, 0)

y200_trainvalid = np.where(y_trainvalid >= 200, 1, 0)
y200_test = np.where(y_test >= 200, 1, 0)
y200_all = np.where(y_all >= 200, 1, 0)

y500_trainvalid = np.where(y_trainvalid >= 500, 1, 0)
y500_test = np.where(y_test >= 500, 1, 0)
y500_all = np.where(y_all >= 500, 1, 0)

y1000_trainvalid = np.where(y_trainvalid >= 1000, 1, 0)
y1000_test = np.where(y_test >= 1000, 1, 0)
y1000_all = np.where(y_all >= 1000, 1, 0)

In [284]:
all_y = [y100_trainvalid, y100_test, y100_all,
         y200_trainvalid, y200_test, y200_all,
         y500_trainvalid, y500_test, y500_all,
         y1000_trainvalid, y1000_test, y1000_all]

y_means = []
for y in all_y: 
    y_means.append(y.mean())

y_names = ['y100_trainvalid', 'y100_test', 'y100_all',
           'y200_trainvalid', 'y200_test', 'y200_all',
           'y500_trainvalid', 'y500_test', 'y500_all',
           'y1000_trainvalid', 'y1000_test', 'y1000_all']

y_df = pd.DataFrame(data=y_means, index=y_names, columns=["Proportion of Ones"])
y_df

,Proportion of Ones
y100_trainvalid,0.654472
y100_test,0.693548
y100_all,0.662338
y200_trainvalid,0.483740
y200_test,0.548387
y200_all,0.496753
y500_trainvalid,0.252033
y500_test,0.258065
y500_all,0.253247
y1000_trainvalid,0.130081


In [287]:
# determine how many 'cases' there are in the y1000_test group - the low number of test cases in this set is
#   an issue in interpreting the AUC below
len(y1000_test) * 0.08

4.96

In [52]:
### Saving the test sets for use in R, or other programs:
# y_names defined above
y_names = ['y100_trainvalid', 'y100_test', 'y100_all',
         'y200_trainvalid', 'y200_test', 'y200_all',
         'y500_trainvalid', 'y500_test', 'y500_all',
         'y1000_trainvalid', 'y1000_test', 'y1000_all']

y_data = [y100_trainvalid, y100_test, y100_all,
         y200_trainvalid, y200_test, y200_all,
         y500_trainvalid, y500_test, y500_all,
         y1000_trainvalid, y1000_test, y1000_all]
file_path = "~/Desktop/Data/Field Data Analysis/Modeling Data Sets from Python/Variable Selection Data Sets/"

for i in np.arange(0, len(y_data)):
    to_save = pd.DataFrame(list(y_data[i]))
    to_save.to_csv(file_path + y_names[i] + ".csv")

In [58]:
# also save the raw y-data
ylog_all = np.log10(y_all + 1)
ylog_all.to_csv(file_path + "ylog_all.csv")

In [102]:
## Variable selection sets:
features_all = ['logSsMean_t1', 'MeanWetness', 'DiffMeanWet', 'MeanWet_1d',
       'MaxWetness', 'DiffMaxWet', 'MaxWet_1d', 'MinWetness', 'DiffMinWet',
       'MinWet_1d', 'MeanTemp', 'DiffMeanT', 'MeanT_1d', 'MaxTemp', 'DiffMaxT',
       'MaxT_1d', 'MinTemp', 'DiffMinT', 'MinT_1d', 'MeanRH', 'DiffMeanRH',
       'MeanRH_1d', 'MaxRH', 'DiffMaxRH', 'MaxRH_1d', 'MinRH', 'DiffMinRH',
       'MinRH_1d', 'DiffRH_0d', 'DiffRH_1d', 'DiffRH_2d', 'MaxDiffRH_2h',
       'MaxDiffRH_3h', 'MeanVPD', 'DiffMeanVPD', 'MeanVPD_1d', 'MaxVPD',
       'DiffMaxVPD', 'MaxVPD_1d', 'MinVPD', 'DiffMinVPD', 'TotalPrecip',
       'Precip_1d', 'RainYN', 'MaxRain', 'MinRain', 'MeanWC', 'DiffMeanWC',
       'MeanWC_1d', 'MaxWC', 'DiffMaxWC', 'MaxWC_1d', 'MinWC', 'DiffMinWC',
       'MinWC_1d', 'MeanDP', 'DiffMeanDP', 'MeanDP_1d', 'MaxDP', 'DiffMaxDP',
       'MaxDP_1d', 'MinDP', 'DiffMinDP', 'MinDP_1d', 'MeanSoilTemp',
       'DiffMeanSoilT', 'MeanSoilT_1d', 'MaxSoilTemp', 'DiffMaxSoilT',
       'MaxSoilT_1d', 'MinSoilTemp', 'DiffMinSoilT', 'MinSoilT_1d', 'RainYN']
# see code below for LASSO feature selection
# this set of variables depends on what the response variable is - i.e. where I set the threshold or whether I use 
#   a continuous response variable. So... for now I'll stick with the continuous response variable 
#   (ylog_all = log(SsMean +1))
features_lasso = ['logSsMean_t1', 'TotalPrecip', 'RainYN', 'DiffMaxWet', 'MinSoilT_1d', 'MeanRH', 
                 'MeanSoilT_1d', 'MeanWC_1d', 'MeanWC', 'DiffMinSoilT', 'DiffMinDP', 'MaxDP_1d', 'DiffRH_1d',
                 'MaxTemp', 'DiffMeanSoilT']
# identified by VIF function in R - found this algorithm on internet
features_VIF = ["logSsMean_t1", "MeanWetness", "MeanWet_1d", "MaxWetness", "MaxWet_1d", "MinWetness",
                "MinWet_1d", "MinTemp", "MinT_1d", "DiffRH_0d", "DiffRH_1d", "DiffRH_2d", "MaxDiffRH_2h",
                "DiffMeanVPD", "TotalPrecip", "Precip_1d", "MeanWC", "DiffMeanWC", "MinWC", "DiffMaxDP",   
                "MaxDP_1d", "DiffMinDP", "MinDP_1d", "DiffMaxSoilT", "MaxSoilT_1d", "DiffMinSoilT",
                "MinSoilT_1d", "RainYN"]
features_biology = ["logSsMean_t1", "MaxTemp", "MeanRH", "MaxDiffRH_3h", "Precip_1d"]
features_no_correlation = ["logSsMean_t1", "DiffMinWet", "MaxTemp", "DiffMaxT", "MaxT_1d", "MaxRH", "DiffMaxRH", "MaxRH_1d", 
                           "MinRH", "DiffMinRH", "MinRH_1d", "MaxDiffRH_2h", "MeanVPD", "DiffMeanVPD", "MeanVPD_1d",
                           "MaxVPD", "DiffMaxVPD", "MaxVPD_1d", "MinVPD", "DiffMinVPD", "MaxRain", "MeanWC", 
                           "DiffMeanWC", "MeanWC_1d", "MaxWC", "DiffMaxWC", "MaxWC_1d", "MinWC", "DiffMinWC", 
                           "MinWC_1d", "MaxDP", "DiffMaxDP", "MaxDP_1d", "MinDP", "DiffMinDP", "MinDP_1d",
                           "MeanSoilTemp", "DiffMeanSoilT", "MeanSoilT_1d", "MaxSoilTemp", "DiffMaxSoilT", 
                           "MaxSoilT_1d", "MinSoilTemp", "DiffMinSoilT", "MinSoilT_1d"]
# omit this approach for now - I don't totally understand the theory/applications behind partial least squares
#    regression
#features_LSR = []
## Same sets as in the R code
features_same_day = ['MeanWetness', 'MaxWetness', 'MinWetness', 'MeanTemp', 'MaxTemp', 'MinTemp',
                      'MeanRH', 'MaxRH', 'MinRH', 'MeanVPD', 'MaxVPD', 'MinVPD', 'TotalPrecip',
                      'RainYN', 'MeanWC', 'MaxWC', 'MinWC', 'MeanDP', 'MaxDP', 'MinDP',
                      'MeanSoilTemp', 'MaxSoilTemp', 'MinSoilTemp']
features_previous_day = ['logSsMean_t1', 'MeanWet_1d', 'MaxWet_1d', 'MinWet_1d', 'MeanT_1d', 'MaxT_1d', 'MinT_1d',
                        'MeanRH_1d', 'MaxRH_1d', 'MinRH_1d', 'DiffRH_1d', 'MeanVPD_1d', 
                        'MaxVPD_1d', 'Precip_1d', 'MeanWC_1d', 'MaxWC_1d', 'MinWC_1d', 'MeanDP_1d',
                        'MaxDP_1d', 'MinDP_1d', 'MeanSoilT_1d', 'MaxSoilT_1d', 'MinSoilT_1d']
features_differences = ['DiffMeanWet', 'DiffMaxWet', 'DiffMinWet', 'DiffMeanT', 'DiffMaxT', 'DiffMinT',
                  'DiffMeanRH', 'DiffMaxRH', 'DiffMinRH', 'DiffRH_0d', 'DiffRH_1d', 'DiffRH_2d',
                  'MaxDiffRH_2h', 'MaxDiffRH_3h', 'DiffMeanVPD', 'DiffMaxVPD', 'DiffMinVPD', 
                  'DiffMeanWC', 'DiffMaxWC', 'DiffMinWC', 'DiffMeanDP', 'DiffMaxDP', 'DiffMinDP',
                  'DiffMeanSoilT', 'DiffMaxSoilT', 'DiffMinSoilT']

In [104]:
feature_sets = [features_all, features_lasso, features_VIF, features_biology, features_no_correlation,
               features_same_day, features_previous_day, features_differences]
feature_set_names = ['features_all', 'features_lasso', 'features_VIF', 'features_biology', 'features_no_correlation',
                     'features_same_day', 'features_previous_day', 'features_differences']

### LASSO feature selection

Prior to LASSO I need to impute and scale my data set

#### NB: I'm going to violate the Golden Rule here by using the whole data set to do the variable selection - then I will use the trainvalid and test data sets to evaluate the model. I'm doing this because my data set is relatively small, and I'll be using which ever methods I determine are best for analysis of data over the next ~2 years (i.e. this is model development mid-way through the research)

In [95]:
numeric_features = ['logSsMean_t1', 'MeanWetness', 'DiffMeanWet', 'MeanWet_1d', 'MaxWetness', 'DiffMaxWet',
       'MaxWet_1d', 'MinWetness', 'DiffMinWet', 'MinWet_1d', 'MeanTemp',
       'DiffMeanT', 'MeanT_1d', 'MaxTemp', 'DiffMaxT', 'MaxT_1d', 'MinTemp',
       'DiffMinT', 'MinT_1d', 'MeanRH', 'DiffMeanRH', 'MeanRH_1d', 'MaxRH',
       'DiffMaxRH', 'MaxRH_1d', 'MinRH', 'DiffMinRH', 'MinRH_1d', 'DiffRH_0d',
       'DiffRH_1d', 'DiffRH_2d', 'MaxDiffRH_2h', 'MaxDiffRH_3h', 'MeanVPD',
       'DiffMeanVPD', 'MeanVPD_1d', 'MaxVPD', 'DiffMaxVPD', 'MaxVPD_1d',
       'MinVPD', 'DiffMinVPD', 'TotalPrecip', 'Precip_1d',  'MeanWC',
       'DiffMeanWC', 'MeanWC_1d', 'MaxWC', 'DiffMaxWC', 'MaxWC_1d', 'MinWC',
       'DiffMinWC', 'MinWC_1d', 'MeanDP', 'DiffMeanDP', 'MeanDP_1d', 'MaxDP',
       'DiffMaxDP', 'MaxDP_1d', 'MinDP', 'DiffMinDP', 'MinDP_1d',
       'MeanSoilTemp', 'DiffMeanSoilT', 'MeanSoilT_1d', 'MaxSoilTemp',
       'DiffMaxSoilT', 'MaxSoilT_1d', 'MinSoilTemp', 'DiffMinSoilT',
       'MinSoilT_1d']
categorical_features = ['RainYN']
drop_features = ['JDay', 'Year', 'SsMean', 'logSsMean', 'logSPCM',]

In [242]:
# create numeric transformer
numeric_transformer = Pipeline([
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler())
])

In [243]:
# create categorical transformer
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first'))
])

In [244]:
# create preprocessor
preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, numeric_features),
    ('categorical', categorical_transformer, categorical_features)
])

In [245]:
preprocessor.fit(asco_trainvalid);

In [246]:
ohe = preprocessor.named_transformers_['categorical'].named_steps['onehot']
ohe_feature_names = list(ohe.get_feature_names(categorical_features))
new_columns = numeric_features + ohe_feature_names
new_columns;

In [247]:
# make data frames of the features
X_trainvalid = pd.DataFrame(preprocessor.transform(asco_trainvalid),  index=asco_trainvalid.index, 
                            columns=new_columns)
X_test = pd.DataFrame(preprocessor.transform(asco_test),  index=asco_test.index,  columns=new_columns)
X_all = pd.DataFrame(preprocessor.transform(asco_2019),  index=asco_2019.index,  columns=new_columns) 

Finally I can do LASSO:

In [59]:
# LASSO feature selection - need scaled features
# ONE OPTION FOR DOING THIS BELOW: this is using the transformed/encoded X data set and using the binary y
#                                  target
# Lasso:
larcv = LassoCV(n_alphas=100, cv = 10, max_iter = 10000) 

#y_data_sets = [y100_trainvalid, y200_trainvalid, y500_trainvalid, y1000_trainvalid]
y_data_sets = [ylog_all, y100_all, y200_all, y500_all, y1000_all]
larcv_coefs = pd.DataFrame(index = X_all.columns)

# a for loop to perform all fits at once:
for y in y_data_sets:
    larcv.fit(X_all, y)
    larcv_ycoefs = pd.DataFrame(data=abs(larcv.coef_), index=X_all.columns)
    
    larcv_coefs = pd.concat([larcv_coefs, larcv_ycoefs], axis=1)
    
larcv_coefs.columns = ['ylog', 'y100', 'y200', 'y500', 'y1000']

In [61]:
# display results; remove variables that weren't selected in any of the models (i.e. their coefficient = 0)
larcv_coefs[(larcv_coefs.T > 1e-6).any()].sort_values(by='ylog', ascending=False)

,ylog,y100,y200,y500,y1000
logSsMean_t1,0.140205,0.077044,0.070383,0.022640,0.015519
TotalPrecip,0.119159,0.060387,0.013555,0.000000,0.000000
RainYN_1.0,0.098137,0.062156,0.000000,0.000000,0.000000
DiffMaxWet,0.073328,0.035758,0.000000,0.000000,0.000000
MinSoilT_1d,0.063239,0.000000,0.000000,0.000000,0.000000
MeanRH,0.040461,0.000000,0.000000,0.045987,0.006877
MeanSoilT_1d,0.032151,0.059527,0.042376,0.000000,0.000000
MeanWC_1d,0.021931,0.020178,0.005226,0.000541,0.000000
MeanWC,0.019865,0.016203,0.014361,0.000000,0.000000
DiffMinSoilT,0.004746,0.013145,0.002026,0.000000,0.000000


Ok, so some similarities between data sets, but mostly pretty different

# Random Forest

The below code is to assess how the scoring functions of `RandomForestClassifier()` work. The first part implements essentially what I'll be doing for the remainder of this analysis (the automated imputation, transformation, and hyperparameter selection) and spits out an AUC value for the test data set. The next section does all those steps manually, uses the the same hyperparameters as the optimal model identified previously, and manually calculates the AUC. Spoiler alert: the AUCs are pretty different between the two methods... why?!

In [296]:
# All steps in the analysis performed at once/automated; the feature set I'm using is the set of features selected
#   by LASSO:

num_features = ['logSsMean_t1', 'TotalPrecip', 'DiffMaxWet', 'MinSoilT_1d', 'MeanRH', 
             'MeanSoilT_1d', 'MeanWC_1d', 'MeanWC', 'DiffMinSoilT', 'DiffMinDP', 'MaxDP_1d', 'DiffRH_1d',
             'MaxTemp', 'DiffMeanSoilT'] 
cat_features = ['RainYN']

# create preprocessors
numeric_transformer = Pipeline([
    ('scaler', StandardScaler()),
    ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
])

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
    ('imputer', SimpleImputer(strategy='most_frequent'))
 ])

preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, num_features),
    ('categorical', categorical_transformer, cat_features)
])

# create rf pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state = 545))
])

# create param_dist dict
param_dist = {
    'model__n_estimators'     : scipy.stats.randint(low=10, high=100),
    'model__max_depth'        : scipy.stats.randint(low=3, high=10),
    'model__class_weight'     : (None, "balanced")
}

# create randomized search CV
random_search_auc = RandomizedSearchCV(rf_pipeline,
                                param_distributions= param_dist,
                                 n_iter = 30,
                                 cv = 10,
                                 scoring = 'roc_auc', # maximize the AUC score (ways to do multiplie criteria)
                                 verbose = 1,
                                 random_state = 454)

random_search_auc.fit(X_trainvalid, y100_trainvalid) 
auc_score = ()
auc_score = random_search_auc.score(X_test, y100_test)
auc_score

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.9min finished


0.6217870257037944

Ok, is this actually doing what I want it to do? Try to rerun the analysis but with me in charge:

In [297]:
# determine the optimal hyperparameters from the RandomizedSearchCV()
random_search_auc.best_params_

{'model__class_weight': 'balanced',
 'model__max_depth': 3,
 'model__n_estimators': 79}

In [204]:
# (unrelated but eventually very useful to have:)
# FINALLY figured out how to get feature importances from the RandomizedSearchCV() pipeline... uggghhhhh
importances = random_search_auc.best_estimator_.steps[1][1].feature_importances_

So now repeat the above but now do it manually, using the same hyperparameters specified as above

In [298]:
# create my own rf object using the above hyperparameters
rf_trial = RandomForestClassifier(class_weight = 'balanced', max_depth = 3, n_estimators = 79, random_state = 545)

In [299]:
num_features = ['logSsMean_t1', 'TotalPrecip', 'DiffMaxWet', 'MinSoilT_1d', 'MeanRH', 
             'MeanSoilT_1d', 'MeanWC_1d', 'MeanWC', 'DiffMinSoilT', 'DiffMinDP', 'MaxDP_1d', 'DiffRH_1d',
             'MaxTemp', 'DiffMeanSoilT'] 
cat_features = ['RainYN'] 

# create preprocessor
numeric_transformer = Pipeline([
    ('scaler', StandardScaler()),
    ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
])

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
    ('imputer', SimpleImputer(strategy='most_frequent'))
 ])

# question: what happens if there are no categorical features? - sort out in the debugging stage
preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, num_features),
    ('categorical', categorical_transformer, cat_features)
])

preprocessor.fit(X_trainvalid)
X_tv_enc = preprocessor.transform(X_trainvalid)
X_test_enc = preprocessor.transform(X_test)

In [300]:
rf_trial.fit(X_tv_enc, y100_trainvalid);

In [301]:
# in the for loop above, I was aiming to maximize roc_auc, to do this manually:
rf_probs = rf_trial.predict_proba(X_test_enc)[:,1]

In [302]:
rf_trial.score(X_test_enc, y100_test)

0.6774193548387096

In [303]:
auc_rf = roc_auc_score(y100_test, rf_probs)
auc_rf

0.6719706242350061

So this value of 0.67 is greater than the value obtained from the CV-method (score = 0.62) - why the difference?!... and even my manual calculation of it is different from the `score` function 

#### Continuing on with the remainder of the analysis...

Here I've written a `for` loop to perform a random forests analysis (hyperparameters determined by `RandomizedSearchCV()`) for the y100 data set. The code performs 7 sets of analysis (one for each subset of features), and is repeated below for each threshold (100, 200, 500, 1000)... eventually this could all be in one loop.

Scoring criteria is for AUC. I wasn't certain this was doing what I wanted it to do, so I eventually included some lines in the `for` loop (see the two iterations of the 1000-threshold loops below) wherein I calculated it manually. The results are similar, but not the same, and I'm not sure why.

In [151]:
# threshold = 100 spores per day

auc_test_scores = []

## NB: data sets called in this loop have not been preprocessed - doing so is part of the pipeline/program
for features in feature_sets:
    # create numeric and categorical feature lists for pipeline - these are based on the selected ones from
    #     above
    num_features = [i for i in features if i in numeric_features] # some function to select numeric features
    cat_features = [i for i in features if i in categorical_features] # if this is a blank list, I believe that's fine
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])
    
    #if len(cat_features) > 0:   # check if cat_features is empty, as this may cause problems in the pipeline
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    # create rf pipeline
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(random_state = 545))
    ])
    # create param_dist dict
    param_dist = {
        'model__n_estimators'     : scipy.stats.randint(low=10, high=100),
        'model__max_depth'        : scipy.stats.randint(low=3, high=10),
        'model__class_weight'     : (None, "balanced")
    }

    # create randomized search CV
    random_search_auc = RandomizedSearchCV(rf_pipeline,
                                    param_distributions= param_dist,
                                     n_iter = 30,
                                     cv = 10,
                                     scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                                     verbose = 1,
                                     random_state = 454)
    random_search_auc.fit(X_trainvalid, y100_trainvalid) 
    auc_score = ()
    auc_score = random_search_auc.score(X_test, y100_test)
    auc_test_scores.append(auc_score)
    
    auc_test_scores

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 13.5min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.4min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.7min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  3.7min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.5min finished


In [152]:
pd.DataFrame(data = auc_test_scores, index = feature_set_names, columns = ['y100'])

,y100
features_all,0.560588
features_lasso,0.621787
features_VIF,0.625459
features_biology,0.410037
features_no_correlation,0.512852
features_same_day,0.555692
features_previous_day,0.521420
features_differences,0.455324


Ok, so pretty bad AUC scores. Interestingly, some are LESS than 0.50 - model fitting portion learning false relationships?

In [153]:
# repeat above, but for 200 ascospores threshold:

auc_test_scores = []

for features in feature_sets:
    # create numeric and categorical feature lists for pipeline - these are based on the selected ones from
    #     above
    num_features = [i for i in features if i in numeric_features] 
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])
    
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    # create rf pipeline
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(random_state = 545))
    ])
    # create param_dist dict
    param_dist = {
        'model__n_estimators'     : scipy.stats.randint(low=10, high=100),
        'model__max_depth'        : scipy.stats.randint(low=3, high=10),
        'model__class_weight'     : (None, "balanced")
    }

    # create randomized search CV
    random_search_auc = RandomizedSearchCV(rf_pipeline,
                                    param_distributions= param_dist,
                                     n_iter = 30,
                                     cv = 10,
                                     scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                                     verbose = 1,
                                     random_state = 454)

    random_search_auc.fit(X_trainvalid, y200_trainvalid) 
    auc_score = ()
    auc_score = random_search_auc.score(X_test, y200_test)
    auc_test_scores.append(auc_score)
    
    auc_test_scores

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 14.2min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.4min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.6min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  3.5min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.3min finished


In [154]:
pd.DataFrame(data = auc_test_scores, index = feature_set_names, columns = ['y200'])

,y200
features_all,0.665441
features_lasso,0.510504
features_VIF,0.513655
features_biology,0.476891
features_no_correlation,0.548319
features_same_day,0.581933
features_previous_day,0.511555
features_differences,0.509454


Again, pretty bad scores here

In [147]:
# as above, for threshold = 500 spores per day

auc_test_scores = []

for features in feature_sets:
    num_features = [i for i in features if i in numeric_features] 
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356)) # was having convergence warnings
                               # when I left 'sample_posterior = False' (the default)
    ])
    
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    # create rf pipeline
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(random_state = 545))
    ])
    # create param_dist dict
    param_dist = {
        'model__n_estimators'     : scipy.stats.randint(low=10, high=100),
        'model__max_depth'        : scipy.stats.randint(low=3, high=10),
        'model__class_weight'     : (None, "balanced")
    }

    # create randomized search CV
    random_search_auc = RandomizedSearchCV(rf_pipeline,
                                    param_distributions= param_dist,
                                     n_iter = 30,
                                     cv = 10,
                                     scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                                     verbose = 1,
                                     random_state = 454)

    random_search_auc.fit(X_trainvalid, y500_trainvalid) 
    auc_score = ()
    auc_score = random_search_auc.score(X_test, y500_test)
    auc_test_scores.append(auc_score)
    
    auc_test_scores

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 13.5min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.2min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 18.1min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  8.1min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.3min finished


In [150]:
pd.DataFrame(data = auc_test_scores, index = feature_set_names, columns = ['y500'])

,y500
features_all,0.692935
features_lasso,0.559783
features_VIF,0.513587
features_biology,0.633152
features_no_correlation,0.591712
features_same_day,0.501359
features_previous_day,0.683424
features_differences,0.586957


In [182]:
# repeat above, but for threshold = 1,000 spores per day:

auc_test_scores = []

for features in feature_sets:
    num_features = [i for i in features if i in numeric_features]
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])

    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    # create rf pipeline
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(random_state = 545))
    ])
    # create param_dist dict
    param_dist = {
        'model__n_estimators'     : scipy.stats.randint(low=10, high=100),
        'model__max_depth'        : scipy.stats.randint(low=3, high=10),
        'model__class_weight'     : (None, "balanced")
    }

    # create randomized search CV
    random_search_auc = RandomizedSearchCV(rf_pipeline,
                                    param_distributions= param_dist,
                                     n_iter = 30,
                                     cv = 10,
                                     scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                                     verbose = 1,
                                     random_state = 454)

    random_search_auc.fit(X_trainvalid, y1000_trainvalid) 
    auc_score = ()
    auc_score = random_search_auc.score(X_test, y1000_test)
    auc_test_scores.append(auc_score)
    
    auc_test_scores

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 13.8min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.5min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  2.7min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  3.7min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.1min finished


In [183]:
pd.DataFrame(data = auc_test_scores, index = feature_set_names, columns = ['y1000'])

,y1000
features_all,0.870175
features_lasso,0.807018
features_VIF,0.578947
features_biology,0.684211
features_no_correlation,0.849123
features_same_day,0.705263
features_previous_day,0.824561
features_differences,0.638596


# Logistic Regression

Here I'll repeat the above, except using logistic regression. For comparison purposes, I will perform a similar analysis in R using more 'statistically sound' assumptions than here.

In [274]:
# Using y100 as the threshold:

y100_auc_test_scores = []
y100_models = {}

for features, name in zip(feature_sets, feature_set_names):
    num_features = [i for i in features if i in numeric_features] # some function to select numeric features
    cat_features = [i for i in features if i in categorical_features] # if this is a blank list, I believe that's fine
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])
    
    #if len(cat_features) > 0:   # check if cat_features is empty, as this may cause problems in the pipeline
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    # question: what happens if there are no categorical features? - sort out in the debugging stage
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    lr_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter = 1000, random_state = 545))
    ])

    # I need to create
    #   a dict of values I want to test in the random search
    param_dist = {
        'model__C'              : (0.0001, 0.001, 0.01, 0.1, 1, 10),
        'model__class_weight'   : (None, 'balanced')
    }

    # Create the GridSearchCV
    grid_search_auc = GridSearchCV(lr_pipeline,
                              param_grid=param_dist,
                              cv = 10,
                              scoring = 'roc_auc', # maximize auc score (ways to do multiplie criteria)
                              verbose = 1)
    
    # obtain AUC values from each of the models
    grid_search_auc.fit(X_trainvalid, y100_trainvalid) 
    auc_score = ()
    auc_score = grid_search_auc.score(X_test, y100_test)
    y100_auc_test_scores.append(auc_score)
    
    # save models in a dict
    model = ()
    model = grid_search_auc.best_estimator_.steps[1][1]
    y100_models.update({name: model})

    y100_auc_test_scores
    y100_models

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.3min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   47.1s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   55.3s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.4min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.6min finished


In [276]:
pd.DataFrame(data = y100_auc_test_scores, index = feature_set_names, columns = ['y100'])

,y100
features_all,0.560588
features_lasso,0.630355
features_VIF,0.614443
features_biology,0.504284
features_no_correlation,0.564259
features_same_day,0.556916
features_previous_day,0.492044
features_differences,0.348837


In [222]:
y100_models['features_all'].coef_

array([[ 0.03069227, -0.00668799,  0.00551904, -0.01388936, -0.00155577,
         0.0033717 , -0.00600539,  0.00674386,  0.00901604, -0.0099779 ,
         0.01100638, -0.00643611,  0.01697477,  0.01182141, -0.00300594,
         0.01473761, -0.00185578, -0.00642211,  0.00536242, -0.01087142,
         0.00414917, -0.01522889, -0.00519809,  0.00693077, -0.01373655,
        -0.00930475,  0.00373999, -0.01317022,  0.0063235 ,  0.00403352,
         0.00095481, -0.0028152 , -0.00834558,  0.01257983, -0.00282413,
         0.01561435,  0.01032244,  0.00064386,  0.00981983,  0.00217304,
        -0.00854457, -0.02000045, -0.00185063,  0.01146107, -0.00854201,
         0.01719726,  0.0048754 , -0.01347158,  0.01558389,  0.01502842,
        -0.00232569,  0.01709551,  0.00110877, -0.00139911,  0.00237499,
         0.00299759, -0.00838348,  0.01050581, -0.00582225, -0.00648019,
         0.00052012,  0.01099685, -0.00492811,  0.01405161,  0.01057914,
        -0.00137551,  0.0116302 ,  0.0056361 , -0.0

Now try for y200:

In [277]:
# as above for threshold = 200 spores per day:

y200_auc_test_scores = []
y200_models = {}

for features, name in zip(feature_sets, feature_set_names):
    num_features = [i for i in features if i in numeric_features] 
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])
    
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    lr_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter = 1000, random_state = 545))
    ])

    # I need to create
    #   a dict of values I want to test in the random search
    param_dist = {
        'model__C'              : (0.0001, 0.001, 0.01, 0.1, 1, 10),
        'model__class_weight'   : (None, 'balanced')
    }

    # Create the GridSearchCV
    grid_search_auc = GridSearchCV(lr_pipeline,
                              param_grid=param_dist,
                              cv = 10,
                              scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                              verbose = 1)
    
    # obtain AUC values from each of the models
    grid_search_auc.fit(X_trainvalid, y200_trainvalid) 
    auc_score = ()
    auc_score = grid_search_auc.score(X_test, y200_test)
    y200_auc_test_scores.append(auc_score)
    
    # save models in a dict
    model = ()
    model = grid_search_auc.best_estimator_.steps[1][1]
    y200_models.update({name: model})

    y200_auc_test_scores
    y200_models

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.3min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   46.5s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   54.5s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.4min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.6min finished


In [278]:
pd.DataFrame(data = y200_auc_test_scores, index = feature_set_names, columns = ['y200'])

,y200
features_all,0.537815
features_lasso,0.568277
features_VIF,0.522059
features_biology,0.504202
features_no_correlation,0.493697
features_same_day,0.540966
features_previous_day,0.536765
features_differences,0.486345


Still pretty bad values here

In [279]:
# Using y500 as the threshold:

y500_auc_test_scores = []
y500_models = {}

for features, name in zip(feature_sets, feature_set_names):
    num_features = [i for i in features if i in numeric_features] 
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])

    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    lr_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter = 1000, random_state = 545))
    ])

    # I need to create
    #   a dict of values I want to test in the random search
    param_dist = {
        'model__C'              : (0.0001, 0.001, 0.01, 0.1, 1, 10),
        'model__class_weight'   : (None, 'balanced')
    }

    # Create the GridSearchCV
    grid_search_auc = GridSearchCV(lr_pipeline,
                              param_grid=param_dist,
                              cv = 10,
                              scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                              verbose = 1)
    
    # obtain AUC values from each of the models
    grid_search_auc.fit(X_trainvalid, y500_trainvalid) 
    auc_score = ()
    auc_score = grid_search_auc.score(X_test, y500_test)
    y500_auc_test_scores.append(auc_score)
    
    # save models in a dict
    model = ()
    model = grid_search_auc.best_estimator_.steps[1][1]
    y500_models.update({name: model})

    y500_auc_test_scores
    y500_models

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.2min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   45.9s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   56.3s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.3min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.5min finished


In [280]:
pd.DataFrame(data = y500_auc_test_scores, index = feature_set_names, columns = ['y500'])

,y500
features_all,0.523098
features_lasso,0.644022
features_VIF,0.523098
features_biology,0.523098
features_no_correlation,0.535326
features_same_day,0.562500
features_previous_day,0.514946
features_differences,0.480978


In [281]:
# as above for threshold = 1,000 spores per day:

y1000_auc_test_scores = []
y1000_models = {}

for features, name in zip(feature_sets, feature_set_names):
    num_features = [i for i in features if i in numeric_features] 
    cat_features = [i for i in features if i in categorical_features] 
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])

    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    lr_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter = 1000, random_state = 545))
    ])

    # I need to create
    #   a dict of values I want to test in the random search
    param_dist = {
        'model__C'              : (0.0001, 0.001, 0.01, 0.1, 1, 10),
        'model__class_weight'   : (None, 'balanced')
    }

    # Create the GridSearchCV
    grid_search_auc = GridSearchCV(lr_pipeline,
                              param_grid=param_dist,
                              cv = 10,
                              scoring = 'roc_auc', # maximize the auc score (ways to do multiplie criteria)
                              verbose = 1)
    
    # obtain AUC values from each of the models
    grid_search_auc.fit(X_trainvalid, y1000_trainvalid) 
    auc_score = ()
    auc_score = grid_search_auc.score(X_test, y1000_test)
    y1000_auc_test_scores.append(auc_score)
    
    # save models in a dict
    model = ()
    model = grid_search_auc.best_estimator_.steps[1][1]
    y1000_models.update({name: model})

    y1000_auc_test_scores
    y1000_models

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.5min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   45.8s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   51.5s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.2min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.4min finished


In [282]:
pd.DataFrame(data = y1000_auc_test_scores, index = feature_set_names, columns = ['y1000'])

,y1000
features_all,0.729825
features_lasso,0.803509
features_VIF,0.719298
features_biology,0.656140
features_no_correlation,0.708772
features_same_day,0.684211
features_previous_day,0.796491
features_differences,0.694737


#### Exploration of how AUC values are calculated:
Try the last model from above, but this time explicitly calculate the AUC to see how they compare:

In [283]:
# Using y1000 as the threshold:

y1000_auc_test_scores_2 = []
calculated_auc = []
y1000_models_2 = {}

for features, name in zip(feature_sets, feature_set_names):
    # create numeric and categorical feature lists for pipeline - these are based on the selected ones from
    #     above
    num_features = [i for i in features if i in numeric_features] # some function to select numeric features
    cat_features = [i for i in features if i in categorical_features] # if this is a blank list, I believe that's fine
    
    # create preprocessor
    numeric_transformer = Pipeline([
        ('scaler', StandardScaler()),
        ('imputer', IterativeImputer(sample_posterior = True, random_state = 356))
    ])
    
    #if len(cat_features) > 0:   # check if cat_features is empty, as this may cause problems in the pipeline
    categorical_transformer = Pipeline([
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='error', drop = 'first')),
        ('imputer', SimpleImputer(strategy='most_frequent'))
     ])
    
    # question: what happens if there are no categorical features? - sort out in the debugging stage
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, num_features),
        ('categorical', categorical_transformer, cat_features)
    ])
    
    lr_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter = 1000, random_state = 545))
    ])

    # I need to create
    #   a dict of values I want to test in the random search
    param_dist = {
        'model__C'              : (0.0001, 0.001, 0.01, 0.1, 1, 10),
        'model__class_weight'   : (None, 'balanced')
    }

    # Create the GridSearchCV
    grid_search_auc = GridSearchCV(lr_pipeline,
                              param_grid=param_dist,
                              cv = 10,
                              scoring = 'roc_auc', # maximize the f1 score (ways to do multiplie criteria)
                              verbose = 1)
    
    # obtain AUC values from each of the models
    grid_search_auc.fit(X_trainvalid, y1000_trainvalid) 
    auc_score = ()
    auc_score = grid_search_auc.score(X_test, y1000_test)
    y1000_auc_test_scores_2.append(auc_score)
    
    AUC = ()
    y_proba = ()
    y_proba = grid_search_auc.predict_proba(X_test)[:,1]
    AUC = roc_auc_score(y1000_test, y_proba)
    calculated_auc.append(AUC)
    
    # save models in a dict
    model = ()
    model = grid_search_auc.best_estimator_.steps[1][1]
    y1000_models_2.update({name: model})

    #y1000_auc_test_scores
    #y1000_models

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  5.6min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   48.2s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   52.2s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.3min finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.7min finished


In [288]:
calculated_auc

[0.7298245614035088,
 0.7929824561403509,
 0.7403508771929824,
 0.656140350877193,
 0.7087719298245614,
 0.6491228070175439,
 0.8210526315789473,
 0.6771929824561403]

In [289]:
y1000_auc_test_scores_2

[0.7298245614035088,
 0.8035087719298246,
 0.7192982456140351,
 0.656140350877193,
 0.7087719298245614,
 0.6842105263157895,
 0.7964912280701755,
 0.6947368421052632]

OK... These are close-ish, but still different. Differences due to randomness somewhere or what?!